In [1]:
import pandas as pd
import numpy as np
import cv2
from skimage.draw import polygon,line

In [2]:
"""https://colab.research.google.com/drive/1AkBpCFU7nW_WLbeKudcLlZD-ZK5DuQCj"""
data_path = ["../data/satellite-data.csv","../data/satellite-data-6d.csv","../data/satellite-data-6m.csv","../data/data-2.csv"][3]

SCALE = 1000
# IRAN
IRAN_TOP_LEFT = (36,44) # long lat
IRAN_TOP_RIGHT = (68,44) # long lat
IRAN_BOTTOM_LEFT = (36,22) # long lat
IRAN_BOTTOM_RIGHT = (68,22) # long lat
IRAN_MIN = IRAN_BOTTOM_LEFT
IRAN_MAX = IRAN_TOP_RIGHT

REGION_MIN = IRAN_BOTTOM_LEFT
REGION_MAX = IRAN_TOP_RIGHT
MAX_X = (REGION_MAX[0]-REGION_MIN[0])*SCALE # longitude
MAX_Y = (REGION_MAX[1]-REGION_MIN[1])*SCALE # latitude
print(MAX_X,MAX_Y)

32000 22000


In [3]:
EARTH_RADIUS = 6371000 # meters
EARTH_CIRCUMFERENCE = 40075160
LAT_TO_METER = EARTH_CIRCUMFERENCE / 360
LON_TO_METER = EARTH_CIRCUMFERENCE / 360
print("LAT_TO_METER:", LAT_TO_METER)
print("LON_TO_METER:", LON_TO_METER)
def lat_to_meter(lat):
    return lat * LAT_TO_METER
def lon_to_meter(lon):
    return lon * LON_TO_METER
def meter_to_lat(meter):
    return (meter / LAT_TO_METER)
def meter_to_lon(meter):
    return meter / LON_TO_METER

LAT_TO_METER: 111319.88888888889
LON_TO_METER: 111319.88888888889


In [4]:
def get_satellite_view_distance(satellite_fov=0.87, satellite_distance=500000):
    earth_radius = EARTH_RADIUS # earth radius in meters
    earth_circumference = EARTH_CIRCUMFERENCE # earth circumference in meters
    earth_max_x = earth_circumference
    earth_max_y = earth_circumference / 2
    satellite_fov_rad = satellite_fov * np.pi / 180 # satellite fov in radians
    satellite_view = 2 * np.tan(satellite_fov_rad) * satellite_distance
    scale_view = satellite_view / 5 # scale view area in meters
    grid_size = scale_view # grid size in meters
    return satellite_view
print(get_satellite_view_distance())

15185.531593941498


In [5]:
def get_point(point,distance,angle,action=1):
    distance = distance * np.sqrt(2)
    if angle<=90 :
        if action%2 == 1:
            theta = (np.pi * (angle-45))/180
            delta_y = np.sin(theta) * distance
            delta_x = np.cos(theta) * distance
            lat = meter_to_lat(delta_y)
            lon = meter_to_lon(delta_x)
            return [point[0]+lon,point[1]+lat] if action == 1 else [point[0]-lon,point[1]-lat]
        elif action%2 == 0:
            theta = (np.pi * (180 - angle - 45))/180
            delta_y = np.sin(theta) * distance
            delta_x = np.cos(theta) * distance
            lat = meter_to_lat(delta_y)
            lon = meter_to_lon(delta_x)
            return [point[0]-lon,point[1]+lat] if action == 2 else [point[0]+lon,point[1]-lat]
    else:
        if action%2 == 1:
            theta = (np.pi * (angle-45))/180
            delta_y = np.sin(theta) * distance
            delta_x = np.cos(theta) * distance
            lat = meter_to_lat(delta_y)
            lon = meter_to_lon(delta_x)
            return [point[0]+lon,point[1]+lat] if action == 1 else [point[0]-lon,point[1]-lat]
        elif action%2 == 0:
            theta = (np.pi * (180 - angle - 45))/180
            delta_y = np.sin(theta) * distance
            delta_x = np.cos(theta) * distance
            lat = meter_to_lat(delta_y)
            lon = meter_to_lon(delta_x)
            return [point[0]-lon,point[1]+lat] if action == 2 else [point[0]+lon,point[1]-lat]

def get_points(distance,point=(0,0),angle=0):
    point1 = get_point(point,distance,angle,1)
    point2 = get_point(point,distance,angle,2)
    point3 = get_point(point,distance,angle,3)
    point4 = get_point(point,distance,angle,4)
    return [point1,point2,point3,point4]

In [235]:
# test
points = get_points(LAT_TO_METER,[0,0],55)
# points = get_points(LAT_TO_METER,[0,0],-55)
print(np.array(points).max())
print(points)

1.3927284806400377
[[1.3927284806400377, 0.2455756079379457], [-0.24557560793794578, 1.3927284806400377], [-1.3927284806400377, -0.2455756079379457], [0.24557560793794578, -1.3927284806400377]]


In [6]:
# read csv data
df = pd.read_csv("../data/data.csv")
df_columns_org = list(df.columns)
df_columns_org

df.drop(df_columns_org[3:], axis=1, inplace=True)

new_columns_names = ["Time","Latitude","Longitude"]
df.columns = new_columns_names
# df = df.drop([i for i in range(6000,df.shape[0])])
print(df.shape)
df.head()

(21446, 3)


,Time,Latitude,Longitude
0,24 Aug 2022 08:02:07.696,44.000,44.565
1,24 Aug 2022 08:02:09.000,43.950,44.650
2,24 Aug 2022 08:02:11.000,43.873,44.781
3,24 Aug 2022 08:02:13.000,43.796,44.912
4,24 Aug 2022 08:02:15.000,43.718,45.042


In [7]:
def filter_region(df,region_min,region_max,limit=3):
    print(region_min,region_max)
    indexes =[]
    for i in range(df.shape[0]):
        point = [df.iloc[i][new_columns_names[2]],df.iloc[i][new_columns_names[1]]]
        if point[0]<=(region_min[0]+limit) or point[0] >= (region_max[0]-limit):
            indexes.append(i)
            continue
        if point[1]<=(region_min[1]+limit) or point[1] >= (region_max[1]-limit):
            indexes.append(i)
            continue
    print(df.shape,len(indexes))
    df = df.drop(indexes)
    return df
    

In [56]:
iran_df = filter_region(df.copy(),REGION_MIN,REGION_MAX,1)
iran_df.head()

(36, 22) (68, 44)
(21446, 3) 3108


,Time,Latitude,Longitude
14,24 Aug 2022 08:02:35.000,42.934,46.325
15,24 Aug 2022 08:02:37.000,42.854,46.451
16,24 Aug 2022 08:02:39.000,42.775,46.577
17,24 Aug 2022 08:02:41.000,42.695,46.703
18,24 Aug 2022 08:02:43.000,42.615,46.828


In [57]:
print(iran_df.max())
print(iran_df.min())

Time         9 Sep 2022 19:22:56.000
Latitude                      42.999
Longitude                     66.997
dtype: object
Time         1 Sep 2022 05:11:32.000
Latitude                      23.005
Longitude                     37.001
dtype: object


In [58]:
iran_df.shape

(18338, 3)

In [59]:
def transform_point(point,scale,region_min,max_x,max_y):
    new_point = []
    new_point.insert(0,(int(point[0] * scale)+max_x/2) - (((region_min[0])*scale)+max_x/2))
    new_point.insert(1,((int(point[1] * scale)+max_y/2) - (((region_min[1])*scale)+max_y/2)))
    return new_point
def transform_points(points,scale,region_min,max_x,max_y):
    new_points = []
    for point in points:
        new_points.append(transform_point(point,scale,region_min,max_x,max_y))
    return new_points

In [60]:
def draw_rectangle(earth,points,scale,region_min,max_x,max_y,step):
    points = transform_points(points,scale,region_min,max_x,max_y)
    points = np.array(points)
    rr, cc = polygon(points[:,0], points[:,1], earth.shape)
    earth[rr,cc] += step
    return earth

def draw_line(earth,point,next_point,scale,region_min,max_x,max_y,step):
    point = transform_point(point,scale,region_min,max_x,max_y)
    next_point = transform_point(next_point,scale,region_min,max_x,max_y)
    rr,cc = line(point[0],point[1],next_point[0],next_point[1])
    earth[rr,cc] += step
    return earth

In [61]:
def fill_earth(df,earth,distance,scale,region_min,angle,max_x,max_y,step=10):
    limit = distance / 2
    print("limit => ",limit)
    for i in range(df.shape[0]-1):
        point = [df.iloc[i][new_columns_names[2]],df.iloc[i][new_columns_names[1]]]
        next_point = [df.iloc[i+1][new_columns_names[2]],df.iloc[i+1][new_columns_names[1]]]
        # angle = angle if int(point[1]) <= int(next_point[1]) else 180 - angle
        points = get_points(limit,point,angle)
        earth = draw_rectangle(earth,points,scale,region_min,max_x,max_y,step)
    return earth   
    

In [14]:
# test transform point
__region_min = [-180,-90]
__max_x = 360000
__max_y = 180000
__scale = 1000
print(transform_point((-40.0,-36.0),__scale,__region_min,__max_x,__max_y))

[140000.0, 54000.0]


In [72]:
earth_map_mat = np.zeros((MAX_X, MAX_Y),dtype=np.uint16)

MemoryError: Unable to allocate 671. MiB for an array with shape (32000, 22000) and data type uint8

In [54]:
distance = get_satellite_view_distance()
earth_map_mat = fill_earth(iran_df,earth_map_mat.copy(),distance,SCALE,REGION_MIN,55,MAX_X,MAX_Y,100)

MemoryError: Unable to allocate 1.31 GiB for an array with shape (32000, 22000) and data type uint16

In [17]:
def analyze_map(map_mat):
    print("mean => ",map_mat.mean())
    print("max => ",map_mat.max())
    print("min => ",map_mat.min())
    print("sum => ",map_mat.sum())

In [41]:
analyze_map(earth_map_mat)

mean =>  47.95750625
max =>  1300
min =>  0
sum =>  3697313328


In [50]:
(earth_map_mat.any()//100)

0

In [ ]:
mat = np.rot90(earth_map_mat//100,1)
cv2.imwrite("earth_map_filter_2.png",mat)

In [55]:
def get_ragion_sumation(earth,points,scale,region_min,max_x,max_y):
    points = transform_points(points,scale,region_min,max_x,max_y)
    points = np.array(points)
    rr, cc = polygon(points[:,0], points[:,1], earth.shape)
    return earth[rr,cc].sum()

def get_shifted_points(point,distance,angle,step):
    if angle <= 90:
        angle = np.pi*(angle/180)
        theta = (np.pi/2) - angle
        delta_x = (np.cos(theta) * distance) * step
        delta_y = (np.sin(theta) * distance) * step
        lon = meter_to_lon(delta_x)
        lat = meter_to_lat(delta_y)
        return [[point[0]+lon,point[1]-lat],[point[0]-lon,point[1]+lat]]
    else:
        angle = np.pi*(angle/180)
        theta = angle - (np.pi/2)
        delta_x = (np.cos(theta) * distance) * step
        delta_y = (np.sin(theta) * distance) * step
        lon = meter_to_lon(delta_x)
        lat = meter_to_lat(delta_y)
        return [[point[0]+lon,point[1]+lat],[point[0]-lon,point[1]-lat]]

def get_rotation_points(earth,point,angle,angle_step,scale,region_min,max_x,max_y):
    distance = get_satellite_view_distance(angle_step)
    print(distance)
    point1,point2 = get_shifted_points(point,distance,angle,1)
    point3,point4 = get_shifted_points(point,distance,angle,2)
    all_points = np.array([
        {
            "point":point3,
            "step":2,
        },
        {
            "point":point1,
            "step":1,
        },
        {
            "point":point,
            "step":0,
        },
        {
            "point":point2,
            "step":-1,
        },
        {
            "point":point4,
            "step":-2,
        }
    ])
    print("all points =>",all_points)
    limit = get_satellite_view_distance()/2
    min_sum = 10e12
    min_point = None
    print("limit => ",limit)
    for pnt in all_points:
        points = get_points(limit,pnt["point"],angle)
        print("points => ",points)
        sumation = get_ragion_sumation(earth,points,scale,region_min,max_x,max_y)
        print("sumation => ",sumation)
        earth = draw_rectangle(earth,points,scale,region_min,max_x,max_y,200)
        if sumation < min_sum:
            min_sum = sumation
            min_point = pnt
    return min_sum,min_point,earth    

In [284]:
min_sum,point,new_map = get_rotation_points(earth_map_mat.copy(),[42.635,24.303],125,0.8,SCALE,REGION_MIN,MAX_X,MAX_Y)

13963.541449181654
all points => [{'point': [42.840502604032025, 24.44689447246495], 'step': 2}
 {'point': [42.73775130201601, 24.374947236232476], 'step': 1}
 {'point': [42.635, 24.303], 'step': 0}
 {'point': [42.53224869798399, 24.231052763767526], 'step': -1}
 {'point': [42.42949739596797, 24.15910552753505], 'step': -2}]
limit =>  7592.765796970749
points =>  [[42.85725251364057, 24.5418879303081], [42.74550914618887, 24.46364438207349], [42.82375269442348, 24.351901014621802], [42.93549606187518, 24.430144562856412]]
sumation =>  1807436
points =>  [[42.75450121162455, 24.469940694075625], [42.642757844172856, 24.391697145841015], [42.721001392407466, 24.279953778389327], [42.83274475985916, 24.358197326623937]]
sumation =>  1918664
points =>  [[42.65174990960854, 24.39799345784315], [42.540006542156846, 24.31974990960854], [42.618250090391456, 24.208006542156852], [42.72999345784315, 24.286250090391462]]
sumation =>  2089452
points =>  [[42.54899860759253, 24.326046221610675], [4

In [265]:
print(min_sum)
print(point)
analyze_map(earth_map_mat)

0
{'point': [42.840502604032025, 24.44689447246495], 'step': 2}
mean =>  1.1424919772727273
max =>  200
min =>  0
sum =>  804314352


In [285]:
img = np.rot90(new_map,1)
cv2.imwrite("earth_map_filter_2.png",img)

True